In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pymorphy2
import re
import numpy as np
from tqdm import tqdm

In [2]:
morph = pymorphy2.MorphAnalyzer()

In [3]:
with open("data/lenta2018.txt", encoding="utf-8") as news_file: # Файл с новостями.
    lenta_news = [n.split("-----\n")[1] for n in news_file.read().split("=====\n")[1:]]
    
with open("data/habr_10000.txt", encoding="utf-8") as news_file: # Файл с новостями.
    habr_news = [n for n in news_file.read().split("=====\n")[1:]]
    

In [4]:
imp_POS = ['ADJF', 'ADJS', 'NOUN', 'VERB', 'PRTF', 'PRTS', 'GRND', 'PREP']


def normalizePymorphy(text):
    tokens = re.findall('[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+', text)
    words = []
    for t in tokens:
        pv = morph.parse(t)
        if str(pv[0].tag.POS) in imp_POS:
            words.append(pv[0].normal_form)
    return words    


In [5]:
lenta_news = [' '.join(normalizePymorphy(news)) for news in tqdm(lenta_news)]
habr_news = [' '.join(normalizePymorphy(news)) for news in tqdm(habr_news)]

100%|█████████████████████████████████████████| 708/708 [00:30<00:00, 23.06it/s]


In [6]:
tfv = TfidfVectorizer(token_pattern="[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+", 
                      ngram_range=(2, 3))

In [7]:
tf_lenta = tfv.fit_transform(lenta_news[:10])

In [9]:
tf_lenta

<10x2729 sparse matrix of type '<class 'numpy.float64'>'
	with 2773 stored elements in Compressed Sparse Row format>

In [10]:
tfv.vocabulary_

{'испытанный украина': 791,
 'украина первый': 2467,
 'первый собственный': 1492,
 'собственный крылатый': 2194,
 'крылатый ракета': 918,
 'ракета создать': 1904,
 'создать кб': 2216,
 'кб луч': 826,
 'луч в': 965,
 'в рамка': 216,
 'рамка нептун': 1914,
 'нептун на': 1217,
 'на основа': 1101,
 'основа российский': 1378,
 'российский противокорабельный': 1985,
 'противокорабельный ракета': 1826,
 'ракета м': 1897,
 'м комплекс': 979,
 'комплекс х': 869,
 'х у': 2560,
 'у о': 2445,
 'о писать': 1251,
 'писать военный': 1516,
 'военный блог': 341,
 'блог который': 90,
 'который вести': 896,
 'вести сотрудник': 305,
 'сотрудник московский': 2245,
 'московский центр': 1052,
 'центр анализ': 2576,
 'анализ стратегия': 34,
 'стратегия технология': 2320,
 'технология напомнить': 2400,
 'напомнить в': 1167,
 'в советский': 239,
 'советский период': 2200,
 'период серийный': 1508,
 'серийный производство': 2138,
 'производство ракета': 1810,
 'м х': 986,
 'х планироваться': 2558,
 'планироватьс

In [11]:
# for news in tf_lenta:
#     print(sorted(news[0][:].todense()[0][0])[-5:])
    
    
freqwords = []
for i in tqdm(range(10)):

    tfs = [(word, tf_lenta[i, index]) for word, index in tfv.vocabulary_.items()
         if tf_lenta[i, index] != 0]
    fw = [w for w, f in sorted(tfs, key = lambda x: x[1], reverse = True)[:5]]
    freqwords.append(fw)


100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 25.16it/s]


In [12]:
freqwords

[['ракета м',
  'собственный крылатый',
  'крылатый ракета',
  'противокорабельный ракета',
  'м комплекс'],
 ['на корточки',
  'русский гопник',
  'русский мат',
  'пользователь выразить',
  'выразить симпатия'],
 ['в результат',
  'сильный удар',
  'в американский',
  'американский штат',
  'штат вирджиния'],
 ['в франция',
  'евро рубль',
  'франция открыться',
  'открыться первый',
  'первый бордель'],
 ['процент россиянин',
  'данный показатель',
  'процент избиратель',
  'на выборы',
  'выборы президент'],
 ['сотрудник фсб',
  'фсб задержать',
  'задержать звезда',
  'звезда телесериал',
  'телесериал детектив'],
 ['на ближний',
  'ближний восток',
  'в регион',
  'совместный предприятие',
  'клиентский сервис'],
 ['причина исчезновение',
  'исчезновение рыболовецкий',
  'рыболовецкий судный',
  'судный восток',
  'восток с'],
 ['в сексуальный',
  'сексуальный домогательство',
  'в сексуальный домогательство',
  'в сша',
  'с оральный'],
 ['американский зоозащитный',
  'зоозащитн

Из теории вероятности используется несколько формул для расчета степени неслучайности для словосочетий.

$$MI(x, y)=log(\frac{f(x,y)*N}{f(x)*f(y)}),$$ 
где $f(x)$ и $f(y)$ - частоты встречаемости слов $x$ и $y$, а $f(x,y)$ - частота встречаемости фразы $x y$.

Для данной метрики берутся слова с самым большими значениями. Однако, так как нас интересует скорее ранжирование слов, чем абсолютные значения метрики, на практике удобнее использовать редуцированную формулу.

$$PMI(x, y)=\frac{f(x,y)}{f(x)*f(y)}$$

Еще одна формула:

$$t-score(x, y)=\frac{f(x,y)-\frac{f(x)*f(y)}{N}}{\sqrt{f(x,y)}},$$
где N - количество слов в коллекции.

О различиях между результатами можно прочитать [здесь](https://cyberleninka.ru/article/n/ot-kollokatsiy-k-konstruktsiyam/viewer) (можно начинать со второго раздела) или [здесь](https://www.researchgate.net/publication/340371729_K_voprosu_o_shodstve_mer_associacii_primenitelno_k_zadace_avtomaticeskogo_izvlecenia_glagolnyh_kollokacij).

Еще можно посмотреть информацию о [c-value](https://www.researchgate.net/publication/220387502_Automatic_Recognition_of_Multi-word_Terms_The_C-value_NC-value_Method).

Давайте посчитаем значения этих 

In [13]:
cntv1 = CountVectorizer(token_pattern="[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+", 
                        ngram_range=(1, 1))
cntv2 = CountVectorizer(token_pattern="[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+", 
                        ngram_range=(2, 2))

In [14]:
cnt_lenta_1 = cntv1.fit_transform(['\n'.join(lenta_news)])
cnt_lenta_2 = cntv2.fit_transform(['\n'.join(lenta_news)])

In [15]:
def calc_MI_tscore(cnt_lenta_1, cnt_lenta_2, cntv1, cntv2, thr):
    mis = {}
    t_scores = {}
    corp_len = cnt_lenta_1.sum()
    for pair, index2 in tqdm(cntv2.vocabulary_.items()):
        freq12 = cnt_lenta_2[0, index2]
        if freq12 < thr:
            continue
        words = pair.split(' ')
        freq1 = cnt_lenta_1[0, cntv1.vocabulary_[words[0]]]
        freq2 = cnt_lenta_1[0, cntv1.vocabulary_[words[1]]]
        mis[pair] = freq12 / (freq1 * freq2)
        t_scores[pair] = (freq12 - (freq1 * freq2) / corp_len) / np.sqrt(freq12)
    return mis, t_scores

In [16]:
mis, t_scores = calc_MI_tscore(cnt_lenta_1, cnt_lenta_2, cntv1, cntv2, 10)

100%|█████████████████████████████████| 139114/139114 [02:09<00:00, 1076.92it/s]


In [17]:
sorted(mis.items(), key=lambda x: x[1], reverse=True)[:100]

[('январярусский смертьзакать', 0.1),
 ('январясадизм какой-тобдсм-маска', 0.1),
 ('февралязолоть стволыонить', 0.1),
 ('упалокто ронять', 0.08333333333333333),
 ('демьян кудрявцев', 0.08333333333333333),
 ('возрождать сверхзвуковой', 0.07692307692307693),
 ('рен тв', 0.07692307692307693),
 ('сверхзвуковой ракетоносец', 0.06993006993006994),
 ('рекс тиллерсон', 0.06666666666666667),
 ('раюдин юсуфов', 0.06666666666666667),
 ('шахабас шахов', 0.06666666666666667),
 ('исаев раюдин', 0.0625),
 ('какой-тобдсм-маска суровый', 0.058823529411764705),
 ('февралякаждый своероссия', 0.058823529411764705),
 ('оливковый ветвь', 0.05803571428571429),
 ('суровый госпожа', 0.053475935828877004),
 ('алина загитов', 0.04807692307692308),
 ('шамиль исаев', 0.046875),
 ('саудовский аравия', 0.045454545454545456),
 ('хайат тахрир', 0.043478260869565216),
 ('настя рыбка', 0.04292929292929293),
 ('тахрир аш-ша', 0.03985507246376811),
 ('джабхата ан-нуср', 0.03764705882352941),
 ('переносный зенитно-ракетный

In [143]:
sorted(t_scores.items(), key=lambda x: x[1], reverse=True)[:100]

[('материал тема', 29.310248678044985),
 ('олимпийский игра', 12.609762051359896),
 ('российский спортсмен', 11.305263157174075),
 ('владимир путин', 10.890176441955774),
 ('тот число', 10.86274528541703),
 ('президент россия', 10.435286936796796),
 ('олимпийский комитет', 10.434333194318143),
 ('риа новость', 10.380572046853729),
 ('игра пхенчхан', 10.196138421758771),
 ('тема декабрь', 10.0570068093562),
 ('уголовный дело', 9.975051441649006),
 ('миллион доллар', 9.836360208198426),
 ('который быть', 9.82751758837726),
 ('февраль сообщать', 9.709321092299627),
 ('миллион рубль', 9.692408505407585),
 ('миллиард доллар', 9.554508232737279),
 ('среда февраль', 9.439329953051642),
 ('прошлое год', 9.413128883623688),
 ('международный олимпийский', 9.274609909692773),
 ('декабрь год', 9.260125332482954),
 ('тема январь', 9.04079617288343),
 ('понедельник февраль', 8.978137866035349),
 ('настоящий время', 8.93791995000938),
 ('вторник февраль', 8.816872246718969),
 ('новость канал', 8.6256

Несколько странные слова в топе в самом деле есть в тексте. Просто надо было при морфологическом анализе посмотреть, что это предсказанные слова, а не словарные.

Вообще, появление этих слов связано с тем, что MI поощряет появление редких слов. Например, если есть два слова, встретившихся вместе 1 раз, причем каждое слово встретилось только один раз (то есть они встретились только вместе), то MI = 1 / 1 * 1 = 1, то есть максимальному значению. Если есть два слова, встречающиеся только вместе, но 100 раз, MI = 100 / 100 * 100 = 0.01.

Посмотрим какие словосочетания встречаются больше 100 раз и какие у них получаются значения метрик.

In [139]:
mis_100, t_scores_100 = calc_MI_tscore(cnt_lenta_1, cnt_lenta_2, cntv1, cntv2, 100)

100%|█████████████████████████████████| 138539/138539 [01:34<00:00, 1458.48it/s]


In [140]:
sorted(mis_100.items(), key=lambda x: x[1], reverse=True)[:100]

[('риа новость', 0.004545071963639425),
 ('владимир путин', 0.0030199213297804847),
 ('уголовный дело', 0.0020567667626491155),
 ('игра пхенчхан', 0.0010347988055465216),
 ('олимпийский комитет', 0.00100116499199068),
 ('материал тема', 0.000997279311739402),
 ('олимпийский игра', 0.0008260985571496008),
 ('тот число', 0.000612948537862342),
 ('российский спортсмен', 0.00041872834439287586),
 ('тема декабрь', 0.00037104692316474483),
 ('президент россия', 0.00016495267564615944),
 ('февраль сообщать', 0.00010707519973643028),
 ('который быть', 3.4707271564828227e-05)]

In [141]:
sorted(t_scores_100.items(), key=lambda x: x[1], reverse=True)[:100]

[('материал тема', 29.310248678044985),
 ('олимпийский игра', 12.609762051359896),
 ('российский спортсмен', 11.305263157174075),
 ('владимир путин', 10.890176441955774),
 ('тот число', 10.86274528541703),
 ('президент россия', 10.435286936796796),
 ('олимпийский комитет', 10.434333194318143),
 ('риа новость', 10.380572046853729),
 ('игра пхенчхан', 10.196138421758771),
 ('тема декабрь', 10.0570068093562),
 ('уголовный дело', 9.975051441649006),
 ('который быть', 9.82751758837726),
 ('февраль сообщать', 9.709321092299627)]

Попробуем теперь избавиться от "странных" слов, получившихся из-за их склейки. PyMorphy2 помечает их, помещая данные о том, как проводился разбор в свойство `methods_stack`. Если слово было разобрано по словарю, будет использован `DictionaryAnalyzer`. Одно из многих значений для предсказанных слов - `FakeDictionary`. 

In [126]:
[type(x[0]) for x in morph.parse('русский')[0].methods_stack]

[pymorphy2.units.by_lookup.DictionaryAnalyzer]

In [133]:
[type(x[0]) for x in morph.parse('январярусский')[0].methods_stack]

[pymorphy2.units.by_analogy.KnownSuffixAnalyzer.FakeDictionary,
 pymorphy2.units.by_analogy.KnownSuffixAnalyzer]

In [125]:
pymorphy2.units.by_analogy.KnownSuffixAnalyzer.FakeDictionary in [type(x[0]) for x in morph.parse('январярусский')[0].methods_stack]

True

Перепишем функцию морфологического анализа так, чтобы она отсеивала несловарные слова (или хотя бы их часть).

In [6]:
imp_POS = ['ADJF', 'ADJS', 'NOUN', 'VERB', 'PRTF', 'PRTS', 'GRND', 'PREP']


def normalizePymorphy2(text):
    tokens = re.findall('[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+', text)
    words = []
    for t in tokens:
        pv = morph.parse(t)
        # Здесь мы проверяем, что слово было предсказано словарем.
        if str(pv[0].tag.POS) in imp_POS and \
          pymorphy2.units.by_lookup.DictionaryAnalyzer in [type(x[0]) for x in pv[0].methods_stack]:
            words.append(pv[0].normal_form)
            
    return words    


In [7]:
lenta_news = [' '.join(normalizePymorphy2(news)) for news in tqdm(lenta_news.split(' '))]
habr_news = [' '.join(normalizePymorphy2(news)) for news in tqdm(habr_news).split(' ')]

100%|█████████████████████████████████████████| 708/708 [00:19<00:00, 36.13it/s]


In [10]:
cntv12 = CountVectorizer(token_pattern="[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+", 
                        ngram_range=(1, 1))
cntv22 = CountVectorizer(token_pattern="[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+", 
                        ngram_range=(2, 2))

In [11]:
cnt_lenta_12 = cntv12.fit_transform(['\n'.join(lenta_news)])
cnt_lenta_22 = cntv22.fit_transform(['\n'.join(lenta_news)])

In [12]:
mis, t_scores = calc_MI_tscore(cnt_lenta_12, cnt_lenta_22, cntv12, cntv22, 10)

100%|█████████████████████████████████| 135212/135212 [01:30<00:00, 1487.83it/s]


In [13]:
sorted(mis.items(), key=lambda x: x[1], reverse=True)[:100]

[('демьян кудрявцев', 0.08333333333333333),
 ('возрождать сверхзвуковой', 0.07692307692307693),
 ('рен тв', 0.07692307692307693),
 ('сверхзвуковой ракетоносец', 0.06993006993006994),
 ('раюдин юсуфов', 0.06666666666666667),
 ('шахабас шахов', 0.06666666666666667),
 ('исаев раюдин', 0.0625),
 ('оливковый ветвь', 0.05803571428571429),
 ('суровый госпожа', 0.053475935828877004),
 ('алина загитов', 0.04807692307692308),
 ('шамиль исаев', 0.046875),
 ('саудовский аравия', 0.045454545454545456),
 ('настя рыбка', 0.04292929292929293),
 ('переносный зенитно-ракетный', 0.03741496598639456),
 ('ксения собчак', 0.03492063492063492),
 ('автомат калашников', 0.034324942791762014),
 ('харви вайнштейн', 0.03333333333333333),
 ('подорвать граната', 0.03213610586011342),
 ('присвоить звание', 0.03205128205128205),
 ('абдусамад гамидов', 0.031862745098039214),
 ('катапультироваться приземлиться', 0.03021978021978022),
 ('прогресс мс', 0.028985507246376812),
 ('святой валентина', 0.02849002849002849),
 (

In [14]:
sorted(t_scores.items(), key=lambda x: x[1], reverse=True)[:100]

[('по тема', 29.025277358715545),
 ('материал по', 28.961915573112304),
 ('один из', 20.637085675507265),
 ('по слово', 19.895021645148802),
 ('в год', 18.621192226259957),
 ('о сообщать', 18.028953646724318),
 ('в время', 15.10094696920619),
 ('в пхенчхан', 13.16388698643111),
 ('по данные', 12.82436070355068),
 ('олимпийский игра', 12.622321641210553),
 ('на сайт', 12.565720730570636),
 ('игра в', 11.605566514111159),
 ('в тот', 11.549181388086014),
 ('российский спортсмен', 11.37166381854365),
 ('в частность', 11.312918715712376),
 ('в россия', 11.014641611104727),
 ('владимир путин', 10.893130181749246),
 ('тот число', 10.877359012498646),
 ('ссылка на', 10.849066547151036),
 ('в результат', 10.834003168804951),
 ('участие в', 10.824299030417833),
 ('с ссылка', 10.595874217708191),
 ('президент россия', 10.48867743985809),
 ('олимпийский комитет', 10.442899332391226),
 ('в конец', 10.414717291660702),
 ('риа новость', 10.382441719165735),
 ('тема декабрь', 10.079480904640327),
 ('к

Расмотрим теперь еще одну метрику - странность (weirdness). Для ее вычисления требуется рассчитать частоты сочетаний выбранной длинны в тематической коллекции и в еще одной - контрастивной, специально подобранном из другой предметной области. После этого рассчитывается отношение частот в тематическом и контрастивной коллекции. Для того, чтобы не делить на 0, применим сглаживание Лагранжа, то есть прибавим 1 к знаменателю.

Так как термины предметной области будут встречаться чаще в тематической коллекции, то самые большие значения будут у слов, похожих на термины. Верно и обратное, так как термины контрастивной коллекции чаще встречаются в ней, то близкие к 0 значения будут означать термины контрастивной коллеции.

In [113]:
hcntv1 = CountVectorizer(token_pattern="[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+", 
                         ngram_range=(1, 1))
hcntv2 = CountVectorizer(token_pattern="[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+", 
                         ngram_range=(2, 2))

In [114]:
cnt_habr_1 = hcntv1.fit_transform(['\n'.join(habr_news)])
cnt_habr_2 = hcntv2.fit_transform(['\n'.join(habr_news)])

In [116]:
weir_1 = {}
weir_2 = {}
for word, index in tqdm(hcntv1.vocabulary_.items()):
    if word in cntv1.vocabulary_.keys():
        w = cnt_habr_1[0, index] / (cnt_lenta_1[0, cntv1.vocabulary_[word]] + 1)
    else:
        w = cnt_habr_1[0, index]
    weir_1[word] = w
    
for word, index in tqdm(hcntv2.vocabulary_.items()):
    if word in cntv2.vocabulary_.keys():
        w = cnt_habr_2[0, index] / (cnt_lenta_2[0, cntv2.vocabulary_[word]] + 1)
    else:
        w = cnt_habr_2[0, index]
    weir_2[word] = w

100%|███████████████████████████████████| 70832/70832 [00:26<00:00, 2703.90it/s]


In [117]:
sorted(weir_1.items(), key=lambda x: x[1], reverse=True)[:100]

[('шрифт', 162),
 ('самохвалов', 153),
 ('граф', 130),
 ('настройка', 96),
 ('евтеев', 90),
 ('плагин', 82),
 ('браузер', 72),
 ('её', 68),
 ('метрика', 64),
 ('префикс', 57),
 ('фреймворк', 52),
 ('интерфейс', 50.5),
 ('оптимизация', 50),
 ('бурладянин', 50),
 ('шаблон', 47),
 ('скрипт', 46.5),
 ('стек', 45),
 ('хабра', 45),
 ('текстура', 45),
 ('трафаретный', 45),
 ('линейный', 44),
 ('буфер', 43),
 ('строка', 42.75),
 ('фича', 42),
 ('построение', 42),
 ('переменный', 38.0),
 ('тестировщик', 37),
 ('актор', 37),
 ('автоматизация', 35),
 ('буква', 34.5),
 ('вершина', 34.333333333333336),
 ('нейронный', 33),
 ('лямбда', 31),
 ('классификация', 30),
 ('преобразование', 30),
 ('иконка', 30),
 ('уза', 30),
 ('гб', 29),
 ('алгоритм', 28.0),
 ('паттерн', 28),
 ('датасет', 28),
 ('лог', 27),
 ('файл', 26.416666666666668),
 ('массив', 24.0),
 ('спецификация', 24),
 ('биткойна', 24),
 ('трекер', 23),
 ('антиква', 23),
 ('стейкхолдер', 23),
 ('репозиторий', 22.0),
 ('кэш', 22),
 ('лоадер', 22)

In [127]:
sorted(weir_2.items(), key=lambda x: x[1], reverse=True)[-100:]

[('год который', 0.1111111111111111),
 ('настоящий момент', 0.1111111111111111),
 ('мужчина женщина', 0.1111111111111111),
 ('деньга быть', 0.1111111111111111),
 ('быть произвести', 0.1111111111111111),
 ('год время', 0.1111111111111111),
 ('компания представить', 0.1111111111111111),
 ('общественный мнение', 0.1111111111111111),
 ('штраф размер', 0.1111111111111111),
 ('европейский союз', 0.1111111111111111),
 ('экономический рост', 0.1111111111111111),
 ('который проводить', 0.1111111111111111),
 ('занять первый', 0.1111111111111111),
 ('миллион человек', 0.1111111111111111),
 ('начаться год', 0.1111111111111111),
 ('финальный этап', 0.1111111111111111),
 ('общий объём', 0.1111111111111111),
 ('год быть', 0.10666666666666667),
 ('конец год', 0.10416666666666667),
 ('наш страна', 0.1),
 ('год этот', 0.1),
 ('стать один', 0.1),
 ('который входить', 0.1),
 ('быть зафиксировать', 0.1),
 ('быть выделить', 0.1),
 ('президент рф', 0.1),
 ('итог год', 0.1),
 ('который заниматься', 0.09090909